In [32]:
#DTW calculation

import pandas as pd
import numpy as np
import os
import glob 
import pickle
from math import *
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from pylab import rcParams
from sklearn import metrics
from sklearn.model_selection import train_test_split
from scipy.ndimage import gaussian_filter
from scipy.fftpack import fft
from dtw import dtw
from numpy import array, zeros, argmin, inf, ndim
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import manhattan_distances
from sklearn import svm
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import accuracy_score
# from quatrotate import qv_mult # this routine implements rotation via quaternion multiplication

%matplotlib inline

In [2]:
def find_contiguous_colors(colors):
    # finds the continuous segments of colors and returns those segments
    segs = []
    curr_seg = []
    prev_color = ''
    for c in colors:
        if c == prev_color or prev_color == '':
            curr_seg.append(c)
        else:
            segs.append(curr_seg)
            curr_seg = []
            curr_seg.append(c)
        prev_color = c
    segs.append(curr_seg) # the final one
    return segs
 
def plot_multicolored_lines(x,y,colors):
    segments = find_contiguous_colors(colors)
    plt.figure(figsize=(20,4))
    start= 0
    for seg in segments:
        end = start + len(seg)
        l, = plt.gca().plot(x[start-1:end+1],y[start-1:end+1],lw=2,c=seg[0]) 
        start = end

In [3]:
# Function to map the colors as a list from the input list of x variables
def color_mapping(lst):
    cols=[]
    colors = {'ST':'blue', 'LT':'red','RT':'green'}
    for l in lst:
        cols.append(colors[l])      
    return cols

In [4]:
def GaussianFilter(df, window_length):
    df1 = df
    
    df1['AX'] = gaussian_filter(df['AX'], window_length)
    df1['AY'] = gaussian_filter(df['AY'], window_length)
    df1['AZ'] = gaussian_filter(df['AZ'], window_length)

    df1['GX'] = gaussian_filter(df['GX'], window_length)
    df1['GY'] = gaussian_filter(df['GY'], window_length)
    df1['GZ'] = gaussian_filter(df['GZ'], window_length)
    
    df1['AX1'] = gaussian_filter(df['AX1'], window_length)
    df1['AY1'] = gaussian_filter(df['AY1'], window_length)
    df1['AZ1'] = gaussian_filter(df['AZ1'], window_length)

    df1['GX1'] = gaussian_filter(df['GX1'], window_length)
    df1['GY1'] = gaussian_filter(df['GY1'], window_length)
    df1['GZ1'] = gaussian_filter(df['GZ1'], window_length)
    return df1

In [5]:
def Amplitude(df, input_features, out_feature):
    df[out_feature] = (df[input_features[0]]**2 + df[input_features[1]]**2  + df[input_features[2]]**2)**(1/2)
  
    

In [6]:
def Yaw_Roll_Pitch(df):
    roll  = []
    pitch = []
    yaw   = []
    for i in range(len(df)):
        x = df['AX'][i]
        y = df['AY'][i]
        z = df['AZ'][i]
        roll1 = atan(y/z)*57.3
        pitch1= atan((-x/(y*y + z*z)**(1/2)))*57.3
        yaw1  = atan((z/((x*x + z*z)**(1/2))))*57.3
        
        roll.append(roll1)
        pitch.append(pitch1)
        yaw.append(yaw1)
    
    df['orientX'] = pitch
    df['orientY'] = roll
    df['orientZ'] = yaw

In [7]:
def getQuat(df):

        """ Given 3 orientation angles, compute the quaternion. """

        yaw   = df['orientZ'] / 2. * np.pi / 180
        roll  = df['orientX'] / 2. * np.pi / 180
        pitch = df['orientY'] / 2. * np.pi / 180

        w =  np.cos(roll) * np.cos(pitch) * np.cos(yaw) + \
                np.sin(roll) * np.sin(pitch) * np.sin(yaw)

        x =  np.sin(roll) * np.cos(pitch) * np.cos(yaw) - \
                np.cos(roll) * np.sin(pitch) * np.sin(yaw)

        y =  np.cos(roll) * np.sin(pitch) * np.cos(yaw) + \
                np.sin(roll) * np.cos(pitch) * np.sin(yaw)

        z =  np.cos(roll) * np.cos(pitch) * np.sin(yaw) - \
                np.sin(roll) * np.sin(pitch) * np.cos(yaw)

        return w, x, y, z

In [8]:
"""
Rotate a 3D vector using the axis-angle method (quaternions).
"""

import numpy as np


def normalize(v, tolerance=0.00001):
    mag2 = sum(n * n for n in v)
    if abs(mag2 - 1.0) > tolerance:
        mag = np.sqrt(mag2)
        v = tuple(n / mag for n in v)
    return v

def q_mult(q1, q2):
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    w = w1 * w2 - x1 * x2 - y1 * y2 - z1 * z2
    x = w1 * x2 + x1 * w2 + y1 * z2 - z1 * y2
    y = w1 * y2 + y1 * w2 + z1 * x2 - x1 * z2
    z = w1 * z2 + z1 * w2 + x1 * y2 - y1 * x2
    return w, x, y, z

def q_conjugate(q):
    w, x, y, z = q
    return (w, -x, -y, -z)

def qv_mult(q1, v1):
    q2 = (0.0,) + v1
    return q_mult(q_mult(q1, q2), q_conjugate(q1))[1:]

def axisangle_to_q(v, theta):
    v = normalize(v)
    x, y, z = v
    theta /= 2
    w = np.cos(theta)
    x = x * np.sin(theta)
    y = y * np.sin(theta)
    z = z * np.sin(theta)
    return w, x, y, z

def q_to_axisangle(q):
    w, v = q[0], q[1:]
    theta = np.acos(w) * 2.0
    return normalize(v), theta

In [9]:
def get_rotation(quatern, vector):
    rotaedvector = []
    for i in range(vector.shape[0]):
        rotaedvector.append(qv_mult(tuple(quatern[i,:]), tuple(vector[i,:])))
    return np.array(rotaedvector)

In [10]:
def rotate(df):
    features = ['A', 'G']
    quaternion = df[['quaternionW', 'quaternionX',
                     'quaternionY','quaternionZ']].values
    
    for f in features:
        xyzlist = [f + 'X', f + 'Y', f + 'Z']
        xyz = df[xyzlist].values
        xyz_rotated = get_rotation(quaternion, xyz)
        df[f + 'X1'] = xyz_rotated[:,0]
        df[f + 'Y1'] = xyz_rotated[:,1]
        df[f + 'Z1'] = xyz_rotated[:,2]
        

In [11]:
#generate frequency domain data from time domain data usinf Fast Fourier transformation

def FFT(df, input_features, output_features):
    
    for i in range(len(input_features)):
        reals = np.real(np.fft.rfft(df[input_features[i]]))
        imagn = np.imag(np.fft.rfft(df[input_features[i]]))

        complexs = [reals[0]]
        n = len(reals)
        if(n%2 == 0):
            complexs.append(imagn[0])
        for j in range(1, n-1):
            complexs.append(reals[j])
            complexs.append(imagn[j])
        if( len(df) > len(complexs)):
            complexs.append(reals[j])
        if( len(df) > len(complexs)):
            complexs.append(imagn[j])
            
        df[output_features[i]] = complexs
    return df

In [12]:
def segmentation(df, N_TIME_STEPS):
    
    N_FEATURES = 1
    step = 50
    segments = []
    labels = []
    for i in range(0, len(df) - N_TIME_STEPS, step):
        gx = df['GX1'].values[i: i + N_TIME_STEPS]
        gy = df['GY1'].values[i: i + N_TIME_STEPS]
        gz = df['GZ1'].values[i: i + N_TIME_STEPS]
        
        ax = df['AX1'].values[i: i + N_TIME_STEPS]
        ay = df['AY1'].values[i: i + N_TIME_STEPS]
        az = df['AZ1'].values[i: i + N_TIME_STEPS]
        
        fax = df['fAX'].values[i: i + N_TIME_STEPS]
        fay = df['fAY'].values[i: i + N_TIME_STEPS]
        faz = df['fAZ'].values[i: i + N_TIME_STEPS]
        
        ma = df['mAcc'].values[i: i + N_TIME_STEPS]
        mz = df['mGyro'].values[i: i + N_TIME_STEPS]
        label = "ST"
        segments.append([gx, gy, gz, ax, ay, az, ma, mz, fax, fay, faz])
#         labels = stats.mode(df['activity'][i: i + N_TIME_STEPS])[0][0]
        labels.append(label)
    return segments, labels

In [13]:
def DTWDistance(s1, s2, w):
    DTW={}

    w = max(w, abs(len(s1)-len(s2)))

    for i in range(-1,len(s1)):
        for j in range(-1,len(s2)):
            DTW[(i, j)] = float('inf')
    DTW[(-1, -1)] = 0

    for i in range(len(s1)):
        for j in range(max(0, i-w), min(len(s2), i+w)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])

    return (DTW[len(s1)-1, len(s2)-1])**(1/2)

In [14]:
def loss_for_segments(segments, comp, feature):
    loss =[]
    labels=[]
    length = len(segments)
    for i in range(length):
        x = segments[i][feature]        
        dist = DTWDistance(x, comp, 50)
#         print(i , '=>', dist)
        loss.append(dist)
    
    return loss

In [15]:
def data_preprocessing(df):
    
    #mean feature of 3-axis accelerometer data
    input_features = ['AX', 'AY', 'AZ']
    output_feature = 'mAcc'
    Amplitude(df, input_features, output_feature)
    
    #mean feature of 3-axis gyroscope data
    input_features = ['GX', 'GY', 'GZ']
    output_feature = 'mGyro'
    Amplitude(df, input_features, output_feature)
    
    #Re-Orientation of 3-axis acc. and gyro sensor data
    Yaw_Roll_Pitch(df)
    qw, qx, qy, qz = getQuat(df)
    df['quaternionW'] = qw
    df['quaternionX'] = qx
    df['quaternionY'] = qy
    df['quaternionZ'] = qz
    rotate(df)
    #apply gaussian filter with window size 10
    df = GaussianFilter(df, 10)

    #Frequency domain feature generation from time series accelerometer data
    input_features = ['AX1', 'AY1', 'AZ1']
    output_feature = ['fAX', 'fAY', 'fAZ']
    df = FFT(df, input_features, output_feature)
    
    #Frequency domain feature generation from time series gyroscope data
    input_features = ['AX1', 'AY1', 'AZ1']
    output_feature = ['fGX', 'fGY', 'fGZ']
    df = FFT(df, input_features, output_feature)
    
    return df

In [16]:
#read csv file and return its data-frame
def read_csv_file(acc, gyro, mode):
    df_gyro = pd.read_csv(gyro, index_col=False)
    df      = pd.read_csv(acc,  index_col=False)
    df.rename(columns = {'X': 'AX', 'Y':'AY', 'Z':'AZ'}, inplace = True)
    df['GX'] = df_gyro['X']
    df['GY'] = df_gyro['Y']
    df['GZ'] = df_gyro['Z']
    
    df['activity'] = mode
    
    return df

In [17]:
def read_files(acc_file, gyro_file, activity):
    df = read_csv_file(acc_file, gyro_file, activity)
    df.reset_index(drop = True , inplace = True)
    if('Milliseconds' in df.columns):
        df = df.drop('Milliseconds', axis=1)
    if('Timestamp' in df.columns):
        df = df.drop('Timestamp', axis=1)

    df = data_preprocessing(df)
    
    return df
    

In [18]:
  
path_acc = os.getcwd() + '/data/bike/*/*/*AccelerometerLinear.csv'  
path_gyro= os.getcwd() + '/data/bike/*/*/*Gyroscope.csv'  
acc_files = glob.glob(path_acc)
gyro_files= glob.glob(path_gyro)
bike_df= read_files(acc_files[0], gyro_files[0], 'bike')
n = len(bike_df)
bike_df = bike_df[500:n-500]
for acc_file, gyro_file in zip(acc_files, gyro_files) :
    print(acc_file)
    print(gyro_file)
    df_this = read_files(acc_file, gyro_file, 'bike')
    n = len(df_this)
    print("len of this file ",df_this.shape)
    df_this = df_this[350 : n-500]
    print("len of this file ",df_this.shape)
    bike_df = pd.concat([bike_df, df_this])
    print(bike_df.shape)
bike_df.reset_index(drop = True, inplace = True)
bike_df.head()


/home/vicky/Desktop/Dhananjay/human-activity/data/bike/Bike0903/2018-09-03_16-06-48/AccelerometerLinear.csv
/home/vicky/Desktop/Dhananjay/human-activity/data/bike/Bike0903/2018-09-03_16-06-48/Gyroscope.csv
len of this file  (302, 28)
len of this file  (0, 28)
(0, 28)
/home/vicky/Desktop/Dhananjay/human-activity/data/bike/Bike0903/2018-09-03_15-58-17/AccelerometerLinear.csv
/home/vicky/Desktop/Dhananjay/human-activity/data/bike/Bike0903/2018-09-03_15-58-17/Gyroscope.csv
len of this file  (11674, 28)
len of this file  (10824, 28)
(10824, 28)
/home/vicky/Desktop/Dhananjay/human-activity/data/bike/Bike0903/2018-09-03_16-12-17/AccelerometerLinear.csv
/home/vicky/Desktop/Dhananjay/human-activity/data/bike/Bike0903/2018-09-03_16-12-17/Gyroscope.csv
len of this file  (10141, 28)
len of this file  (9291, 28)
(20115, 28)
/home/vicky/Desktop/Dhananjay/human-activity/data/bike/Bike0903/2018-09-03_15-53-27/AccelerometerLinear.csv
/home/vicky/Desktop/Dhananjay/human-activity/data/bike/Bike0903/2018-

,AX,AY,AZ,GX,GY,GZ,activity,mAcc,mGyro,orientX,...,AZ1,GX1,GY1,GZ1,fAX,fAY,fAZ,fGX,fGY,fGZ
0,-0.007023,0.112130,-0.430526,-0.133173,-0.076351,0.104627,bike,1.033584,0.355847,-19.020191,...,-0.339794,-0.109928,-0.013069,0.113804,25.880820,146.803098,-86.776934,25.880820,146.803098,-86.776934
1,-0.003683,0.139261,-0.417439,-0.141457,-0.077701,0.103369,bike,1.199937,0.379878,-17.621220,...,-0.319991,-0.114300,-0.011571,0.113162,32.421071,52.301557,-21.304957,32.421071,52.301557,-21.304957
2,0.000853,0.167799,-0.401824,-0.148915,-0.078951,0.101645,bike,1.686179,0.369426,2.516505,...,-0.297427,-0.118204,-0.010350,0.112090,73.125050,-23.648178,47.068088,73.125050,-23.648178,47.068088
3,0.006639,0.197498,-0.383849,-0.155505,-0.080184,0.099483,bike,0.843485,0.343799,-2.775456,...,-0.272269,-0.121649,-0.009453,0.110630,-36.111748,93.802537,-5.337681,-36.111748,93.802537,-5.337681
4,0.013667,0.228047,-0.363736,-0.161207,-0.081489,0.096916,bike,2.172636,0.342391,82.421936,...,-0.244741,-0.124649,-0.008932,0.108831,2.880542,97.677857,42.925926,2.880542,97.677857,42.925926


In [19]:
print(bike_df.shape)
bike_df.columns

(55575, 28)


Index(['AX', 'AY', 'AZ', 'GX', 'GY', 'GZ', 'activity', 'mAcc', 'mGyro',
       'orientX', 'orientY', 'orientZ', 'quaternionW', 'quaternionX',
       'quaternionY', 'quaternionZ', 'AX1', 'AY1', 'AZ1', 'GX1', 'GY1', 'GZ1',
       'fAX', 'fAY', 'fAZ', 'fGX', 'fGY', 'fGZ'],
      dtype='object')

In [20]:
bike_segments, bike_labels = segmentation(bike_df, 400)
np.array(bike_segments).shape

(1104, 11, 400)

In [21]:
path_acc = os.getcwd() + '/data/car1/*/*/*AccelerometerLinear.csv'  
path_gyro= os.getcwd() + '/data/car1/*/*/*Gyroscope.csv'
acc_files = glob.glob(path_acc)
gyro_files= glob.glob(path_gyro)
car_df= read_files(acc_files[0], gyro_files[0], 'car')
n = len(car_df)
car_df = car_df[350:n-500]
for acc_file, gyro_file in zip(acc_files, gyro_files) :
    print(acc_file)
    print(gyro_file)
    df_this = read_files(acc_file, gyro_file, 'car')
    n = len(df_this)
    print("len of this file ",df_this.shape)
    df_this = df_this[350 : n-500]
    print("len of this file ",df_this.shape)
    car_df = pd.concat([car_df, df_this])
    print(car_df.shape)
car_df.reset_index(drop = True, inplace = True)
car_df.head()

/home/vicky/Desktop/Dhananjay/human-activity/data/car1/car/2018-08-17_16-43-04/AccelerometerLinear.csv
/home/vicky/Desktop/Dhananjay/human-activity/data/car1/car/2018-08-17_16-43-04/Gyroscope.csv
len of this file  (6646, 28)
len of this file  (5796, 28)
(11592, 28)
/home/vicky/Desktop/Dhananjay/human-activity/data/car1/car/2018-08-17_16-54-34/AccelerometerLinear.csv
/home/vicky/Desktop/Dhananjay/human-activity/data/car1/car/2018-08-17_16-54-34/Gyroscope.csv
len of this file  (325, 28)
len of this file  (0, 28)
(11592, 28)
/home/vicky/Desktop/Dhananjay/human-activity/data/car1/car/2018-08-17_16-45-30/AccelerometerLinear.csv
/home/vicky/Desktop/Dhananjay/human-activity/data/car1/car/2018-08-17_16-45-30/Gyroscope.csv
len of this file  (6770, 28)
len of this file  (5920, 28)
(17512, 28)
/home/vicky/Desktop/Dhananjay/human-activity/data/car1/car/2018-08-17_16-54-46/AccelerometerLinear.csv
/home/vicky/Desktop/Dhananjay/human-activity/data/car1/car/2018-08-17_16-54-46/Gyroscope.csv
len of thi

,AX,AY,AZ,GX,GY,GZ,activity,mAcc,mGyro,orientX,...,AZ1,GX1,GY1,GZ1,fAX,fAY,fAZ,fGX,fGY,fGZ
0,-0.274150,0.386046,-0.116917,0.034840,0.137716,0.332758,car,0.882817,0.233839,-63.984609,...,0.181182,0.091135,0.031582,0.245277,22.431667,-34.025393,8.622702,22.431667,-34.025393,8.622702
1,-0.250386,0.374052,-0.113326,0.035408,0.116555,0.339397,car,1.370899,0.218887,23.101936,...,0.174576,0.086128,0.024991,0.244226,-0.893364,12.487488,-4.808171,-0.893364,12.487488,-4.808171
2,-0.226391,0.362938,-0.109220,0.035289,0.096402,0.342103,car,1.577567,0.223861,51.300015,...,0.165708,0.080448,0.019058,0.241016,4.278284,-14.240124,-1.744896,4.278284,-14.240124,-1.744896
3,-0.202447,0.352584,-0.104442,0.034548,0.077432,0.341075,car,2.392651,0.637923,58.367552,...,0.154992,0.074300,0.013742,0.235883,0.723327,-32.780075,-41.302828,0.723327,-32.780075,-41.302828
4,-0.178788,0.342868,-0.098848,0.033261,0.059779,0.336601,car,0.679046,0.835866,-47.974722,...,0.142876,0.067882,0.008987,0.229098,8.278835,-24.641124,19.177460,8.278835,-24.641124,19.177460


In [22]:
print(car_df.shape)
car_df.columns

(49269, 28)


Index(['AX', 'AY', 'AZ', 'GX', 'GY', 'GZ', 'activity', 'mAcc', 'mGyro',
       'orientX', 'orientY', 'orientZ', 'quaternionW', 'quaternionX',
       'quaternionY', 'quaternionZ', 'AX1', 'AY1', 'AZ1', 'GX1', 'GY1', 'GZ1',
       'fAX', 'fAY', 'fAZ', 'fGX', 'fGY', 'fGZ'],
      dtype='object')

In [83]:
car_segments, car_labels = segmentation(car_df, 400)
print(np.array(car_segments).shape)
print(np.array(bike_segments).shape)

(978, 11, 400)
(1104, 11, 400)


In [84]:
# car_seg = [*zip(*(car_segments[]))]
car_seg = np.transpose(car_segments, axes=(0, 2, 1)).tolist()
bike_seg = np.transpose(bike_segments, axes=(0, 2, 1)).tolist()
# seg = [list(x) for x in zip(*car_segments)]

In [85]:
print(np.array(car_seg).shape)
print(np.array(bike_seg).shape)

label1 = [1]*len(car_seg)     #lebel = 1 to all bike samples

label2 = [0]*len(bike_seg)
label = label1 + label2
print(len(label))

(978, 400, 11)
(1104, 400, 11)
2082


In [86]:
seg = car_seg + bike_seg

In [87]:
seg = pd.Series(seg)
label = pd.Series(label)

In [88]:

RANDOM_SEED = 42
X_train, X_test, y_train, y_test = train_test_split(seg, label, test_size=0.2, random_state=RANDOM_SEED)

In [90]:
X_train.shape

(1665,)

In [91]:
# Input Data 

training_data_count = len(X_train)  # 7352 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 2947 testing series
n_steps = len(X_train[0])  # 128 timesteps per series
n_input = len(X_train[0][0])  # 9 input parameters per timestep

In [92]:
print(training_data_count, test_data_count,n_steps, n_input)

1665 417 400 11


In [93]:
n_hidden = 16 # Hidden layer num of features
n_classes = 2 # Total classes

In [96]:

# Training 

learning_rate = 0.0025
lambda_loss_amount = 0.0015
training_iters = training_data_count * 300  # Loop 300 times on the dataset
batch_size = 100
display_iter = 30  # To show test set accuracy during training


# Some debugging info

print("Some useful info to get an insight on dataset's shape and normalisation:")
print("(X shape, y shape, every X's mean, every X's standard deviation)")
print(X_test.shape, y_test.shape, np.mean(X_test.tolist), np.std(X_test.tolist))
print("The dataset is therefore properly normalised, as expected, but not yet one-hot encoded.")

Some useful info to get an insight on dataset's shape and normalisation:
(X shape, y shape, every X's mean, every X's standard deviation)


TypeError: unsupported operand type(s) for /: 'method' and 'int'

In [68]:
def LSTM_RNN(_X, _weights, _biases):
    # Function returns a tensorflow LSTM (RNN) artificial neural network from given parameters. 
    # Moreover, two LSTM cells are stacked which adds deepness to the neural network. 
    # Note, some code of this notebook is inspired from an slightly different 
    # RNN architecture used on another dataset, some of the credits goes to 
    # "aymericdamien" under the MIT license.

    # (NOTE: This step could be greatly optimised by shaping the dataset once
    # input shape: (batch_size, n_steps, n_input)
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    # Reshape to prepare input to hidden activation
    _X = tf.reshape(_X, [-1, n_input]) 
    # new shape: (n_steps*batch_size, n_input)
    
    # ReLU activation, thanks to Yu Zhao for adding this improvement here:
    _X = tf.nn.relu(tf.matmul(_X, _weights['hidden']) + _biases['hidden'])
    # Split data because rnn cell needs a list of inputs for the RNN inner loop
    _X = tf.split(_X, n_steps, 0) 
    # new shape: n_steps * (batch_size, n_hidden)

    # Define two stacked LSTM cells (two recurrent layers deep) with tensorflow
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
    # Get LSTM cell output
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)

    # Get last time step's output feature for a "many-to-one" style classifier, 
    # as in the image describing RNNs at the top of this page
    lstm_last_output = outputs[-1]
    
    # Linear activation
    return tf.matmul(lstm_last_output, _weights['out']) + _biases['out']


def extract_batch_size(_train, step, batch_size):
    # Function to fetch a "batch_size" amount of data from "(X|y)_train" data. 
    
    shape = list(_train.shape)
    shape[0] = batch_size
    batch_s = np.empty(shape)

    for i in range(batch_size):
        # Loop index
        index = ((step-1)*batch_size + i) % len(_train)
        batch_s[i] = _train[index] 

    return batch_s


def one_hot(y_, n_classes=n_classes):
    # Function to encode neural one-hot output labels from number indexes 
    # e.g.: 
    # one_hot(y_=[[5], [0], [3]], n_classes=6):
    #     return [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    
    y_ = y_.reshape(len(y_))
    return np.eye(n_classes)[np.array(y_, dtype=np.int32)]  # Returns FLOATS

In [69]:
# Graph input/output
x = tf.placeholder(tf.float32, [None, n_steps, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

# Graph weights
weights = {
    'hidden': tf.Variable(tf.random_normal([n_input, n_hidden])), # Hidden layer weights
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=1.0))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

pred = LSTM_RNN(x, weights, biases)

# Loss, optimizer and evaluation
l2 = lambda_loss_amount * sum(
    tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()
) # L2 loss prevents this overkill neural network to overfit the data
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred)) + l2 # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer

correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



KeyboardInterrupt: 

In [ ]:
# To keep track of training's performance
test_losses = []
test_accuracies = []
train_losses = []
train_accuracies = []

# Launch the graph
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)

# Perform Training steps with "batch_size" amount of example data at each loop
step = 1
while step * batch_size <= training_iters:
    batch_xs =         extract_batch_size(X_train, step, batch_size)
    batch_ys = one_hot(extract_batch_size(y_train, step, batch_size))

    # Fit training using batch data
    _, loss, acc = sess.run(
        [optimizer, cost, accuracy],
        feed_dict={
            x: batch_xs, 
            y: batch_ys
        }
    )
    train_losses.append(loss)
    train_accuracies.append(acc)
    
    # Evaluate network only at some steps for faster training: 
    if (step*batch_size % display_iter == 0) or (step == 1) or (step * batch_size > training_iters):
        
        # To not spam console, show training accuracy/loss in this "if"
        print("Training iter #" + str(step*batch_size) + \
              ":   Batch Loss = " + "{:.6f}".format(loss) + \
              ", Accuracy = {}".format(acc))
        
        # Evaluation on the test set (no learning made here - just evaluation for diagnosis)
        loss, acc = sess.run(
            [cost, accuracy], 
            feed_dict={
                x: X_test,
                y: one_hot(y_test)
            }
        )
        test_losses.append(loss)
        test_accuracies.append(acc)
        print("PERFORMANCE ON TEST SET: " + \
              "Batch Loss = {}".format(loss) + \
              ", Accuracy = {}".format(acc))

    step += 1

print("Optimization Finished!")

# Accuracy for test data

one_hot_predictions, accuracy, final_loss = sess.run(
    [pred, accuracy, cost],
    feed_dict={
        x: X_test,
        y: one_hot(y_test)
    }
)

test_losses.append(final_loss)
test_accuracies.append(accuracy)

print("FINAL RESULT: " + \
      "Batch Loss = {}".format(final_loss) + \
      ", Accuracy = {}".format(accuracy))